# 16S Sequencing Clean-up

In [ ]:
%%bash
#Import Raw Sequence data into qiime
qiime tools import   
--type MultiplexedSingleEndBarcodeInSequence  \
--input-path seqs/   \
--output-path multiplexed_seqs.qza

#Demultiplex sequences (use barcodes to determine which sequences belong to which sample)
qiime cutadapt demux-single  \
--i-seqs multiplexed_seqs.qza  \
--m-barcodes-file metadata.tsv   \
--m-barcodes-column barcodes   \
--p-error-rate 0   \
--o-per-sample-sequences demultiplexed-seqs.qza   \
--o-untrimmed-sequences untrimmed.qza

#Check quality of sequences
qiime demux summarize \
  --i-data demultiplexed-seqs.qza \
  --o-visualization demux.qzv

mkdir trim250

#Trim sequences to include sequences where mean is ~>30 (determined by looking at demux.qzv)
qiime dada2 denoise-single \
--i-demultiplexed-seqs demultiplexed-seqs.qza \
--p-trim-left 0 --p-trunc-len 250 \
--o-representative-sequences trim250/repseqs.qza \
--o-table trim250/table.qza \
--o-denoising-stats stats-dada2.qza

# 16S Taxonomy and filtering

In [3]:
%%bash

wget https://data.qiime2.org/2022.8/common/silva-138-99-515-806-nb-classifier.qza


#Assign Taxonomy using silva classifier
qiime feature-classifier classify-sklearn \
--i-classifier silva-138-99-515-806-nb-classifier.qza \
--i-reads trim250/repseqs.qza \
--o-classification trim250/taxonomy.qza

cd trim250/

#create quick barplot for visualization and to see what should be cut out
qiime taxa barplot \
--i-table table.qza \
--i-taxonomy taxonomy.qza \
--m-metadata-file 16S_metadata.tsv \
--o-visualization taxabarplot.qzv

#Create taxa feature table to assign taxonomy (exclude mitochondria and chloroplasts)
qiime taxa filter-table --i-table table.qza -\
-i-taxonomy taxonomy.qza \
--p-exclude mitochondria,chloroplast,Unassigned,Eukaryota \
--o-filtered-table filtertaxa.qza

mkdir filtering; cd filtering


#Run Adrian's code to determine how many samples a feature must be in to be kept (4.15 in this case)
#Note you need to export the frequency table then convert to a .tsv for Adrian's code to work
#Then remove those samples in QIIME2
qiime tools export \
--input-path ../filtertaxa.qza \
--output-path . 

biom convert -i feature-table.biom -o 16Sfeaturetable.tsv --to-tsv

python3 ../calccuttoff.py -ct ./16Sfeaturetable.tsv

qiime feature-table filter-features \
--i-table ../ex-mito_ex-chloro.qza \
--p-min-samples 4  \
--o-filtered-table freqfiltered.qza


#Remove positive/negative controls
awk -F '\t' '{print $1}' ../16S_metadata.tsv  > sampleids.txt
grep -v "P\|N" sampleids.txt > samples_to_keep.tsv

qiime feature-table filter-samples \
--i-table freqfiltered.qza \
--m-metadata-file samples_to_keep.tsv \
--o-filtered-table fully-filt.qza

#export features 
qiime tools export \
--input-path fully-filt.qza \
--output-path fexported

biom convert -i feature-table.biom -o 16S-features.tsv --to-tsv

#export classified features
qiime taxa collapse \
--i-table fully-filt.qza \
--i-taxonomy ../taxonomy.qza \
--p-level 6 \
--o-collapsed-table 16Sfilteredtable.qza

qiime tools export \
--input-path 16Sfilteredtable.qza \
--output-path texported

biom convert -i feature-table.biom -o 16S-taxatable.tsv --to-tsv

# 16S Basic Analysis

In [ ]:
%%bash
#Create Tree
qiime phylogeny align-to-tree-mafft-fasttree \
--i-sequences repseqs.qza \
--o-alignment aligned-repseqs.qza \
--o-masked-alignment masked-align-repseqs.qza \
--o-tree unrooted-tree.qza \
--o-rooted-tree rooted-tree.qza

#Calculate core diversity metrics
qiime diversity core-metrics-phylogenetic \
--i-phylogeny rooted-tree.qza \
--i-table fully-filt.qza \
--p-sampling-depth 1000 \
--m-metadata-file 16S_metadata.tsv \
--output-dir core-metrics

#Faith's PD Alpha Diversity
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics/faith_pd_vector.qza \
--m-metadata-file 16S_metadata.tsv \
--o-visualization core-metrics/faithpd-group-sig.qzv

#Evenness Alpha Diversity
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics/evenness_vector.qza \
--m-metadata-file 16S_metadata.tsv \
--o-visualization core-metrics/evenness-group-sig.qzv

#Shannon Alpha Diversity
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics/shannon_vector.qza \
--m-metadata-file 16S_metadata.tsv \
--o-visualization core-metrics/shannon-group-sig.qzv

#Chao1 Richness
qiime diversity alpha \
--i-table core-metrics/rarefied_table.qza \
--p-metric 'chao1' \
--o-alpha-diversity core-metrics/chao1_vector.qza

qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics/chao1_vector.qza \
--m-metadata-file 16S_metadata.tsv \
--o-visualization core-metrics/chao1-group-sig.qzv

# ITS Sequencing Clean-up

In [ ]:
%%bash
qiime tools import   \
--type MultiplexedSingleEndBarcodeInSequence   \
--input-path seqs/   \
--output-path multiplexed_seqs.qza

qiime cutadapt demux-single   \
--i-seqs multiplexed_seqs.qza   \
--m-barcodes-file its_metadata.tsv   \
--m-barcodes-column barcodes   \
--p-error-rate 0   \
--o-per-sample-sequences demultiplexed-seqs.qza   \
--o-untrimmed-sequences untrimmed.qza

qiime demux summarize \
--i-data demultiplexed-seqs.qza \
--o-visualization demultiplexed-seqs.qza

qiime dada2 denoise-single \
--i-demultiplexed-seqs demultiplexed-seqs.qza \
--p-trim-left 0 \
--p-trunc-len 240 
--o-representative-sequences trim240/repseqs.qza \
--o-table trim240/table.qza \
--o-denoising-stats stats-dada2.qza\


# Unite Database Training

In [ ]:
%%bash
wget https://files.plutof.ut.ee/public/orig/C5/54/C5547B97AAA979E45F79DC4C8C4B12113389343D7588716B5AD330F8BDB300C9.tgz
tar xzf C5547B97AAA979E45F79DC4C8C4B12113389343D7588716B5AD330F8BDB300C9.tgz
cd sh_qiime_release_10.05.2021/developer/
awk '/^>/ {print($0)}; /^[^>]/ {print(toupper($0))}' sh_refs_qiime_ver8_99_10.05.2021_dev.fasta | tr -d ' ' > sh_refs_qiime_ver8_99_10.05.2021_dev_uppercase.fasta

qiime tools import \
--type FeatureData[Sequence] \
--input-path sh_refs_qiime_ver8_99_10.05.2021_dev.fasta \
--output-path unite-ver8-seqs_99_10.05.2021.qza

qiime tools import \
--type FeatureData[Taxonomy] \
--input-path sh_taxonomy_qiime_ver8_99_10.05.2021_dev.txt \
--output-path unite-ver8-taxonomy_99_10-05-2021.qza \
--input-format HeaderlessTSVTaxonomyFormat

qiime feature-classifier fit-classifier-naive-bayes 
--i-reference-reads unite-ver8-seqs_99_10.05.2021.qza 
--i-reference-taxonomy unite-ver8-taxonomy_99_10-05-2021.qza 
--o-classifier classifier.qza

qiime feature-classifier classify-sklearn \
--i-classifier ../classifier.qza \
--i-reads repseqs.qza \
--o-classification its_taxonomy.qza

# ITS Filtering Data

In [ ]:
%%bash
cd trim240/

#Summarize feature table to see if some samples have very low counts
#this data looks good. Nothing super low except for the negative control, which will be removed
mkdir filt2; cd filt2

qiime tools export \
--input-path ../table.qza \
--output-path .

biom convert -i feature-table.biom -o itsfeattable.tsv --to-tsv

python3 ../../calccuttoff.py ./itsfeattable.tsv #tells us to remove things with up to 4 zeros

#remove features with zeros in less than 5 samples
qiime feature-table filter-features \
--i-table ../table.qza \
--p-min-samples 5 \
--o-filtered-table filt2/freqfilt.qza

awk -F '\t' '{print $1}' ../../its_metadata.tsv  > sampleids.txt
grep -v "P\|N" sampleids.txt > samples_to_keep.tsv

qiime feature-table filter-samples \
--i-table freqfilt.qza \
--m-metadata-file samples_to_keep.tsv \
--o-filtered-table its-fully-filt.qza

cd .. 

qiime tools export \
--input-path filt2/its-fully-filt.qza \
--output-path fexport

cd fexport/
biom convert -i feature-table.biom -o itsfeaturetable.tsv --to-tsv

#collapse table to genus level and export taxa table
qiime taxa collapse \
--i-table table.qza \
--i-table its_taxonomy.qza
--p-level 6 \
--o-collapsed-table its-taxa.qza

qiime tools export --input-path its_taxa.qza --output-path texport

cd texport/
biom convert -i feature-table.biom -o itstaxatable.tsv --to-tsv

# ITS Basic Analysis

In [ ]:
%%bash

#create quick taxa barplot
qiime taxa barplot \
--i-table table.qza \
--i-taxonomy its_taxonomy.qza \
--m-metadata-file ../its_metadata.tsv 
--o-visualization its_taxa_barplot.qzv

cp filt/its-fully-filt.qza ./
mkdir diversity-metrics

qiime diversity alpha-rarefaction \
--i-table its-fully-filt.qza \
--p-max-depth 10000 \
--p-metrics 'pielou_e' 'chao1' 'shannon' \
--m-metadata-file ../its_metadata.tsv \
--o-visualization diversity-metrics/rarefaction.qzv

qiime diversity core-metrics \
--i-table its-fully-filt.qza \
--p-sampling-depth 3500 \
--m-metadata-file ../its_metadata.tsv \
--output-dir core-metrics

mv core-metrics/ diversity-metrics/

qiime diversity alpha-group-significance \
--i-alpha-diversity shannon_vector.qza \
--m-metadata-file ../../../its_metadata.tsv \
--o-visualization ad_shannon.qzv

qiime diversity alpha-group-significance -\
-i-alpha-diversity evenness_vector.qza \
--m-metadata-file ../../../its_metadata.tsv \
--o-visualization ad_evenness.qzv

cd ..

qiime diversity alpha \
--i-table core-metrics/rarefied_table.qza \
--p-metric 'chao1' \
--o-alpha-diversity chao1_vector.qza

qiime diversity alpha-group-significance \
--i-alpha-diversity chao1_vector.qza \
--m-metadata-file ../../its_metadata.tsv \
--o-visualization ad_chao1.qzv